# Autocombat TOP and StrokeMRI vs. EDIS harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.seperated as sep
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
EDIS = pd.read_csv('../new_data/TrainingDataComplete_EDIS.csv')
# HELIUS = pd.read_csv('../new_data/TrainingDataComplete_HELIUS.csv')
# SABRE = pd.read_csv('../new_data/TrainingDataComplete_SABRE.csv')
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')
#Insight46 = pd.read_csv('../new_data/TrainingDataComplete_Insight46.csv')

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
EDIS = EDIS.drop(['ID'],axis = 1)
EDIS.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
datasets = [TOPMRI, EDIS]
new_frames = sep.deal_with_readout_and_labelling(datasets, ['M0'])

In [ ]:
# make sure things are inproper order
for frame in new_frames:
    print(len(frame))
print(len(TOPMRI))
print(len(EDIS))

In [ ]:
#new_frames[0]
TOPMRI = new_frames[0]
EDIS = new_frames[1]


In [ ]:
new_frame_datasets=  [TOPMRI,  EDIS, ]
for everyone in new_frame_datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRIEDIS = pd.concat([TOPMRI, EDIS])
TOPMRIEDIS

In [ ]:
# make sure things are inproper order
for frame in new_frames:
    print(len(frame))
print(len(TOPMRI))
print(len(EDIS))

In [ ]:
#new_frames[0]
TOPMRI = new_frames[0]
EDIS = new_frames[1]


In [ ]:
new_frame_datasets=  [TOPMRI,  EDIS, ]
for everyone in new_frame_datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRIEDIS['decade']=(TOPMRIEDIS['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRIEDIS = TOPMRIEDIS.sort_values(by='age')
TOPMRIEDIS.reset_index(inplace=True)
TOPMRIEDIS['fine_grain'] = TOPMRIEDIS['age'].rolling(2).sum()/2
TOPMRIEDIS

In [ ]:
TOPMRIEDIS[2:].fine_grain.iloc[::2] = TOPMRIEDIS[:].fine_grain.iloc[1::2]
TOPMRIEDIS['fine_grain']

In [ ]:
TOPMRIEDIS['fine_grain'][0] = TOPMRIEDIS['fine_grain'][1]
TOPMRIEDIS['fine_grain']

In [ ]:
TOPMRIEDIS['decade'].unique()

In [ ]:
TOPMRIEDIS.columns

In [ ]:
combat = autocombat.Combat(
    features=[# 'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
             #'ld', 'pld', 'labelling', 'readout',],
    sites=["site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ #'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
            # 'ld', 'pld', 'labelling', 'readout',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRIEDIS))
print(fg_combat.fit(TOPMRIEDIS))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRIEDIS= combat.transform(TOPMRIEDIS)
transformed_TOPMRIEDIS.head(3)

In [ ]:
# check what harmonization did
for column in ['age', 'sex', 'site', 'gm_vol', 'wm_vol',
       'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
       'ld', 'pld', 'labelling', 'readout', 'decade', 'fine_grain']:
        
    print(column, sum(TOPMRIEDIS[column] - transformed_TOPMRIEDIS[column]))

In [ ]:
fg_transformed_TOPMRIEDIS= fg_combat.transform(TOPMRIEDIS)
fg_transformed_TOPMRIEDIS.head(3)

In [ ]:
TOPMRIEDIS.head(3)

In [ ]:
transformed_TOPMRIEDIS= combat.transform(TOPMRIEDIS)
transformed_TOPMRIEDIS.head(3)

In [ ]:
fg_transformed_TOPMRIEDIS= fg_combat.transform(TOPMRIEDIS)
fg_transformed_TOPMRIEDIS.head(3)

In [ ]:
transformed_TOPMRIEDIS.columns

In [ ]:
TOPMRI_transformed = transformed_TOPMRIEDIS[ transformed_TOPMRIEDIS['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_TOPMRIEDIS[ fg_transformed_TOPMRIEDIS['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
EDIS_transformed = transformed_TOPMRIEDIS[ transformed_TOPMRIEDIS['site']==1]
EDIS_transformed = EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
EDIS_transformed.head(3)

In [ ]:
fg_EDIS_transformed = fg_transformed_TOPMRIEDIS[fg_transformed_TOPMRIEDIS['site']==1]
fg_EDIS_transformed = fg_EDIS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_EDIS_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/autocom_harm_topmri_v_e.csv')
EDIS_transformed.to_csv('harm_results/autocombat/autocom_harm_EDIS.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_topmri_v_e.csv')
fg_EDIS_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_EDIS1.csv')

# Data quality check

In [ ]:
TOPMRI_transformed.isna().sum().sum()

In [ ]:
EDIS_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]

In [ ]:
#neuro_harm_top.head(3)
(TOPMRI_transformed[number_columns] < 0).sum()

In [ ]:
(EDIS_transformed[number_columns] < 0).sum()

In [ ]:
EDIS_transformed

## So some variables are again problematic, becuas they are negative

 We will wait about logging it until we see other columns we may want to log across all datasets